<a href="https://colab.research.google.com/github/rondrori/floodmap/blob/main/floodplainV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Detecting floodplains from topography using
[pyflwdir](https://github.com/Deltares/pyflwdir)
based on this [example](https://github.com/Deltares/pyflwdir/blob/main/examples/elevation_indices.ipynb)

1. Read subset of DTM file.
2. Calculate flow direction.
3. Calculate upstream area.
4. calculate height above nearest drain (HAND) using [Nobre et al (2016)](https://doi.org/10.1002/hyp.10581)
5. Delineate floodplains using [Nardi et al (2019)](http://www.doi.org/10.1038/sdata.2018.309)

In [ ]:
# Install libs
!pip install pyflwdir
!pip install rasterio
!pip install cartopy>=0.20
!pip install descartes
!pip install geopandas>=0.8
!pip install matplotlib
!pip install rasterio


In [7]:
# Imports
import math
import rasterio
import pyflwdir
import matplotlib.pyplot as plt
from rasterio.merge import merge
from rasterio.plot import show
import glob
import os

In [8]:
# Config ---

# DATA - dont forget to connect drive
file = '/content/drive/MyDrive/MAPI/DTM_Israel_10m_2017_2022.tif'

# blocksize to read (geotiff block size 512)
blkSize = 512

# how many blocks in tile
blkFactor = 10

# tile size
tileSize = blkSize * blkFactor

# flood threshokd
floodTrsh = 250


In [9]:
# Read file properties

with rasterio.open(file, "r") as src:
    nodata = src.nodata
    transform = src.transform
    crs = src.crs
    prf = src.profile

width  = prf['width']
height = prf['height']
src.close()


In [10]:
# asssign UL corner and resulotion
Xscale = prf['transform'][0]
X0 = prf['transform'][2]
Yscale = prf['transform'][4]
Y0 = prf['transform'][5]

# LR corner
X1 = X0 + width*Xscale
Y1 = Y0 + height*Yscale


In [11]:
# calculate UL corner for tile
def ULtile(X0, Y0, x, y, tileSize):
  Xmin = X0 + Xscale * x * tileSize
  Xmax = Xmin + tileSize * Xscale
  Ymax = Y0 + Yscale * y * tileSize
  Ymin =  Ymax + tileSize * Yscale
  return Xmin, Xmax, Ymin, Ymax

In [ ]:
# calculate number of tiles in each dimession.
# geotif is written with rotation (X=height, Y=width)
X = math.floor(width/blkSize) + 1
Y = math.floor(height/blkSize) + 1

# loop over tiles, read and process each
for x in range(X):
  for y in range(Y):


    # calculate current tile corners
    Xmin, Xmax, Ymin, Ymax = ULtile(X0, Y0, x, y, tileSize)

    # calculate window indices
    rowmin = y*tileSize
    rowmax = rowmin + tileSize
    colmin = x*tileSize
    colmax = colmin + tileSize

    if rowmax > height:
      rowmax = height
    if colmax > width:
      colmax = width

    # read tile
    with rasterio.open(file, "r") as src:
      #elevtn = src.read(1,window=((rowmin, rowmax), (colmin, colmax)))
      elevtn = src.read(1,window=((rowmin, rowmax), (colmin, colmax)))
      prf = src.profile
    src.close()
    trns = rasterio.Affine(10.0, 0.0, Xmin,0.0, -10.0, Ymax)
    try:
      flw = pyflwdir.from_dem(
          data=elevtn,
          nodata=src.nodata,
          transform=trns,
          latlon=crs.is_geographic,
        )
      # first we derive the upstream area map
      uparea = flw.upstream_area("km2")
      hand = flw.hand(drain=uparea > floodTrsh, elevtn=elevtn)
      floodplains = flw.floodplains(elevtn=elevtn, uparea=uparea, upa_min=floodTrsh)

      prf.update({'transform': trns, 'width': tileSize, 'height': tileSize, 'dtype': 'int8',  'nodata': -1})
      f = f'/content/drive/MyDrive/flow/flood_{x}_{y}_{floodTrsh}.tif'
      print(f"writing: {f}")
      with rasterio.open(f, 'w', **prf) as dst:
        dst.write_band(1, floodplains)
      dst.close()
    except Exception as e:
    # Print the error message
      print(f"An error occurred: {e}")
      continue


# Define the directory containing the raster files
raster_dir = "/content/drive/MyDrive/flow/"


# List all raster files
raster_files = glob.glob(os.path.join(raster_dir, f"flood*_{floodTrsh}.tif"))

# Open the raster files
src_files_to_mosaic = []
for fp in raster_files:
    src = rasterio.open(fp)
    src_files_to_mosaic.append(src)

# Merge the rasters
mosaic, out_trans = merge(src_files_to_mosaic)

# Update the metadata
out_meta = src.meta.copy()
out_meta.update({
    "driver": "GTiff",
    "height": mosaic.shape[1],
    "width": mosaic.shape[2],
    "transform": out_trans,
    "crs": src.crs
})

# Define the output file path
output_file = f"/content/drive/MyDrive/flow/flood_{floodTrsh}_.tif"

# Write the merged raster to disk
with rasterio.open(output_file, "w", **out_meta) as dest:
    dest.write(mosaic)

# Optionally, display the merged raster
#show(mosaic, cmap='terrain')
